In [1]:
from binary_param import linear, constant
import pandas as pd
import numpy as np
from R32_bmimPF6_PR import configuration
from pyomo.environ import (ConcreteModel,
                           SolverFactory,
                           units as pyunits)

import idaes.logger as idaeslog
import pyomo.contrib.parmest.parmest as parmest

import pytest

from idaes.core import FlowsheetBlock
from idaes.generic_models.unit_models import Flash
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

from idaes.core.util.model_statistics import degrees_of_freedom

In [2]:
def PR_model(data):
    """
    Function which returns initilized model.

    Args:
        data: pandas DataFrame with data

    Returns:
        initialized model
    """
    eps = 0.1
    init_kappa_2_1A = -0.1
    init_kappa_1_2A = -0.05
    m = ConcreteModel()

    m.fs = FlowsheetBlock(default={"dynamic": False})

    m.fs.properties = GenericParameterBlock(default=configuration)

    m.fs.state_block = m.fs.properties.state_block_class(
        default={"parameters": m.fs.properties,
                 "defined_state": True})
    x = float(data['x_R32'])+eps
    m.fs.state_block.flow_mol.fix(1)
    m.fs.state_block.temperature.fix(float(data["temperature"]))
    m.fs.state_block.pressure.fix(float(data["pressure"]))
    m.fs.state_block.mole_frac_comp['bmimPF6'].fix(1-x)
    m.fs.state_block.mole_frac_comp['R32'].fix(x)

    # parameter - kappa_A_ij (set at 0.3, 0 if i=j)
    m.fs.properties.PR_kappa_A['bmimPF6','bmimPF6'].fix(0)
    m.fs.properties.PR_kappa_A['bmimPF6', 'R32'].fix(init_kappa_2_1A)
    m.fs.properties.PR_kappa_A['R32', 'R32'].fix(0)
    m.fs.properties.PR_kappa_A['R32', 'bmimPF6'].fix(init_kappa_1_2A)
    m.fs.properties.PR_kappa_B['bmimPF6', 'bmimPF6'].fix(0)
    m.fs.properties.PR_kappa_B['bmimPF6', 'R32'].fix(0)
    m.fs.properties.PR_kappa_B['R32', 'R32'].fix(0)
    m.fs.properties.PR_kappa_B['R32', 'bmimPF6'].fix(0)
    m.fs.properties.PR_kappa_C['bmimPF6', 'bmimPF6'].fix(0)
    m.fs.properties.PR_kappa_C['bmimPF6', 'R32'].fix(0)
    m.fs.properties.PR_kappa_C['R32', 'R32'].fix(0)
    m.fs.properties.PR_kappa_C['R32', 'bmimPF6'].fix(0)
    # Initialize the flash unit
    m.fs.state_block.initialize(outlvl=idaeslog.CRITICAL)

    # Fix the state variables on the state block
    m.fs.state_block.pressure.unfix()
    m.fs.state_block.mole_frac_comp['bmimPF6'].unfix()
    m.fs.state_block.mole_frac_comp['R32'].unfix()
    m.fs.state_block.temperature.fix(float(data["temperature"]))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'R32'].fix(float(data['x_R32']))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'bmimPF6'].fix(float(data['x_bmimPF6']))
    m.fs.state_block.mole_frac_comp['R32'].fix(float(data['x_R32'])+eps)
    m.fs.state_block.mole_frac_comp['bmimPF6'].unfix()

    # Set bounds on variables to be estimated
    m.fs.properties.PR_kappa_A['bmimPF6', 'R32'].setlb(-5)
    m.fs.properties.PR_kappa_A['bmimPF6', 'R32'].setub(5)

    m.fs.properties.PR_kappa_A['R32', 'bmimPF6'].setlb(-5)
    m.fs.properties.PR_kappa_A['R32', 'bmimPF6'].setub(5)

    # Return initialized flash model
    return m

def SSE(m, data):
    """
    returns objective function expresion.

    Args:
        m: model
        data: pandas DataFrame with data

    Returns:
        objectuve function scaled expresion
    """
    expr = ((float(data["pressure"]) - m.fs.state_block.pressure)**2)

    return expr * 1e-6

variable_name = ["fs.properties.PR_kappa_A['bmimPF6', 'R32']",
                 "fs.properties.PR_kappa_A['R32', 'bmimPF6']"]

#     pest = parmest.Estimator(PR_model, data, variable_name, SSE, tee=True)

#     obj_value, parameters,a= pest.theta_est(calc_cov=True)

In [3]:
data = pd.read_csv('R32IL.csv')

# print(data)


In [4]:
N = 10 #number of samples
results_params = np.zeros((N,2))
results_obj = np.zeros((N,1))

st_dev_T = 0.01 #C
st_dev_P = 9771 #MPa
st_dev_x = 0.07 #mole fraction

for i in range(N):
    print(i)
#     noise_T = np.random.normal(loc=0,scale=st_dev_T)
#     print(noise_T)
#     noise_P = np.random.normal(loc=0,scale=st_dev_P)
#     print(noise_P)
    noise_x = np.random.normal(loc=0,scale=st_dev_x)
    print(noise_x)
    data_ = data.copy()
#     data_["temperature"] += noise_T
    data_["x_bmimPF6"] += data_["x_bmimPF6"] * noise_x
#     data_["pressure"] += noise_P
    
#     data_.loc[data_['pressure'] <= 0, 'pressure'] = 1000
    data_.loc[data_['x_bmimPF6'] <= 0.0001, 'x_bmimPF6'] = 0.0001
    data_.loc[data_['x_bmimPF6'] >= 0.9999 , 'x_bmimPF6'] = 0.9999      

    try:
        pest = parmest.Estimator(PR_model, data_, variable_name, SSE, tee=False)

        obj_value, parameters = pest.theta_est()

        for k,v in parameters.items():
            print(k, "=", v)

        line = 0

        for k,v in parameters.items():
            results_params[i,line] = v
            line += 1
        results_obj[i] = obj_value

    except ValueError:
        results_params[i,0] = 'NaN'
        results_params[i,1] = 'NaN'


0
-0.02374179504274429
2021-10-28 12:57:56 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 12:57:57 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 12:57:59 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 12:58:00 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 12:58:02 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 12:58:04 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 12:58:05 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 12:58:07 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-2

2021-10-28 13:00:58 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:01:00 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:01:02 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:01:05 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:01:07 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:01:09 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:01:11 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:01:13 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:01:16 [INFO] idaes

2021-10-28 13:03:46 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:03:48 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:03:50 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:03:52 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:03:56 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:03:58 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:03:59 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:04:01 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:04:03 [INFO] idaes

2021-10-28 13:06:57 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:07:02 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:07:04 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:07:05 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:07:07 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:07:13 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:07:18 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:07:22 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:07:26 [INFO] idaes

2021-10-28 13:09:50 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:09:52 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:09:53 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:09:55 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:09:56 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:09:58 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:09:59 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:10:00 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:10:02 [INFO] idaes

In [5]:
print(results_params)

[[-0.11469335 -0.05685016]
 [-0.13258351 -0.06553241]
 [-0.12252417 -0.06050592]
 [-0.11063498 -0.05503433]
 [-0.11332188 -0.05623078]
 [-0.10841425 -0.05406191]
 [-0.12057399 -0.05957578]
 [-0.10038154 -0.0506528 ]
 [-0.13178477 -0.06511839]
 [-0.13686685 -0.06780041]]


In [6]:
results = pd.DataFrame(results_params)
results.to_csv('R32_bmim_7.csv')